###Canvas Creation from Enrichr

In [8]:
import pandas as pd 
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib
from matplotlib.ticker import MaxNLocator
from IPython.display import display,FileLink, Markdown
from matplotlib import colors

In [37]:
bar_color = 'mediumforestgreen'
bar_color_not_sig = 'lightgrey'
edgecolor = None
chart_type = "Canvas"
all_libraries = ['ARCHS4_Cell-lines']

# list of genes that was input
genes = ['TP53', 'TNF', 'EGFR', 'GKN1', 'HADHA', 'APOE', 'ESR1', 'VEGFA', 'TGFB1', 'PREPL', 'TIA1', 'TPO', 'TTN', 'SATB2', 'CHPF', 'MALL', 'MIPIP', 'NUPL1']

# need to download relevant Enrichr library. Then, the number of terms in that library is the number of hexagons we want. Each term has a certain number of genes associated with it, we compute the Jaccard index for that set and the input gene set, then color the respective hexagon. 

# NOTE: currently only covers case where these is one library
raw_library_data = []
library_data = []

for library_name in all_libraries:
    with open('library.txt', 'r') as f:
        for line in f.readlines():
            raw_library_data.append(line.split("\t\t"))

name = []
gene_list = []

for i in range(len(raw_library_data)):
    name += [raw_library_data[i][0]]
    raw_genes = raw_library_data[i][1].split("\t")
    gene_list += [raw_genes[:-1]]

library_data = [list(a) for a in zip(name, gene_list)]

# now we have library_data formatted in the way we want: a 2D list where the first element is the name and the second element is a list of genes associated with that name

jaccard_indices = []
indices = []

for gene_set in library_data:
    intersection = [value for value in gene_set[1] if value in genes]
    index = len(intersection)/(len(gene_set[1]) + len(genes))
    jaccard_indices += [[gene_set[0], index]]
    indices += [index]
#print(jaccard_indices)
#print(jaccard_indices[0:10])

#add step here that takes in color input
color_list = []
#print(max(indices))

# indices[i]*10, indices[i]*5, indices[i]*5

for i in range(256):
    if i < len(indices):
        color_list += [matplotlib.colors.to_hex((indices[i]*5, indices[i]*10, indices[i]*5))]
    else:
        color_list += ["#000000"]

#print(color_list)

f = open("index_colors.txt", "w")
for hex_color in color_list:
    f.write(hex_color + "\n")
f.close()

##Single canvas from a single library

#Steps to creating a canvas:  
1. Open downloaded library from Enrichr  
2. Compute the Jaccard index between all sets in the library  
3. Applying simulated annealing to determine where the hexagons should end up  
4. Color the hexagons based on their index  
5. Celebrate.

In [38]:
import uuid
from textwrap import dedent
from IPython.core.display import display, HTML
from string import Template



In [39]:
# NOTE: your d3 code should basically be a function, in the future it would be best to take it out.

def init_chart():
  #javascript = 'element.append("{}");'.format(str(color_list))
  chart_id = 'mychart-' + str(uuid.uuid4())
  display(HTML('<script src="/static/components/requirejs/require.js"></script>'))
  display(HTML(Template(dedent('''
  <script>
  require.config({
    paths: {
      'd3': 'https://cdnjs.cloudflare.com/ajax/libs/d3/5.16.0/d3.min',
      'd3-hexbin': 'https://d3js.org/d3-hexbin.v0.2.min',
    },
    shim: {
      'd3-hexbin': ['d3']
    }
  })

  // If we configure mychart via url, we can eliminate this define here
  define($chart_id, ['d3', 'd3-hexbin'], function(d3, d3_hexbin) {
    return function (figure_id, numA, numB, colorList) {
      var margin = {top: 50, right: 20, bottom: 20, left: 50},
        width = 850 - margin.left - margin.right,
        height = 350 - margin.top - margin.bottom;

      // append the svg object to the body of the page
      var svG = d3.select('#' + figure_id)
        .attr("width", width + margin.left + margin.right)
        .attr("height", height + margin.top + margin.bottom)
        .append("g")
          .attr("transform",
                "translate(" + margin.left + "," + margin.top + ")");
      
      //The number of columns and rows of the heatmap
      var MapColumns = numA,
          MapRows = numB;

      //The maximum radius the hexagons can have to still fit the screen
      var hexRadius = d3.min([width/((MapColumns + 0.5) * Math.sqrt(3)), height/((MapRows + 1/3) * 1.5)]);

      //Calculate the center position of each hexagon
      var points = [];
      for (var i = 0; i < MapRows; i++) {
          for (var j = 0; j < MapColumns; j++) {
              var x = hexRadius * j * Math.sqrt(3)
              //Offset each uneven row by half of a "hex-width" to the right
              if(i%2 === 1) x += (hexRadius * Math.sqrt(3))/2
              var y = hexRadius * i * 1.5
              points.push([x,y])
          }
      }

      //Set the hexagon radius
      var hexbin = d3_hexbin.hexbin().radius(hexRadius);

      svG.append("g")
        .selectAll(".hexagon")
        .data(hexbin(points))
        .enter().append("path")
        .attr("class", "hexagon")
        .attr("d", function (d) {
            return "M" + d.x + "," + d.y + hexbin.hexagon();
        })
        .attr("stroke", "white")
        .attr("stroke-width", "1px")
        .style("fill", function (d,i) { return colorList[i]; })
    }
  })
  </script>
  ''')).substitute({ 'chart_id': repr(chart_id) })))
  return chart_id

def Canvas(numA, numB, colorList):
  chart_id = init_chart()
  display(HTML(Template(dedent('''
  <svg id=$figure_id></svg>
  <script>
  require([$chart_id], function(mychart) {
    mychart($figure_id, $numA, $numB, $colorList)
  })
  </script>
  ''')).substitute({
      'chart_id': repr(chart_id),
      'figure_id': repr('fig-' + str(uuid.uuid4())),
      'numA': repr(numA),
      'numB': repr(numB),
      'colorList': repr(color_list)
  })))

In [40]:
Canvas(16, 16, color_list)